# there is a known issue in Canvas import that empty folders are not imported
# https://community.canvaslms.com/t5/Archived-Questions/ARCHIVED-Duplicating-folder-structures/m-p/142963

# this script will find out the missing empty folders by comparing the source Canvas site with target Canvas site
# and add those missing folders with its parent folders recursively

In [ ]:
# install the UCF canvasapi library
# https://github.com/ucfopen/canvasapi
!pip install canvasapi

In [4]:
# all the configuration variables needed for this script

# this step is for Google Colab setup;
# use the Key button on the left panel and enter your key named after "CANVAS_API_KEY"
from google.colab import userdata
# init canvas api instance with token and url
API_KEY = userdata.get('CANVAS_API_KEY')
# Canvas instance url
API_URL = "canvas url"

# get canvas assignments with giving ids
import json
import os
from canvasapi import Canvas

# init canvas api instance
canvas = Canvas(API_URL, API_KEY)

In [67]:
# Canvas course ids for source and target courses
source_course_id = source_site_id
target_course_id = target_course_id

In [ ]:
source_course = canvas.get_course(source_course_id)
print(source_course)
# create a set
source_folders = set()
for folder in source_course.get_folders():
    # add folder to set
    source_folders.add(folder.full_name)
# print source_folders size
print("count of folders:" + len(source_folders))

In [ ]:
target_course = canvas.get_course(target_course_id)
print(target_course)
# create a set
target_folders = set()
for folder in target_course.get_folders():
    # add folder to set
    target_folders.add(folder.full_name)
# print source_folders size
print("count of folders=" + len(target_folders))

In [ ]:
# compare source and target folders
# put the diff in a list and sort it
diff_folders = list(source_folders - target_folders)
diff_folders.sort()
# print each element inside diff_folders, use list comprehension
[print(folder) for folder in diff_folders]

# print the size of diff_folders
print("count of missing folders:" + len(diff_folders))

In [ ]:
# get target course root folder
folders = target_course.resolve_path()
root_folder = None
for folder in folders:
    print(folder)
    root_folder = folder
    break

In [71]:
# create folder recursively based on the folder path
def create_folder_recursive(folder_path, parent_folder):
    folder_path = folder_path.replace('course files/', '')
    # split the folder path
    folders = folder_path.split("/")
    # iterate over each folder
    for folder in folders:
        found_folder = False
        # check if folder exists
        for f in parent_folder.get_folders():
            if f.name == folder:
                # update boolean
                found_folder = True
                # update parent folder
                parent_folder = f
                break
        if not found_folder:
            # create folder
            parent_folder.create_folder(folder)

In [ ]:
# for each folder inside the diff_folders,
# based on the folder name, find the parent folder name by parsing its path
# find whether the parent folder exist in the target course
# if not, create the parent folder
# then create the folder in the target course
# if parent folder exist, create the folder in the target course
# if the folder already exist, print the folder name
count = 0
for folder in diff_folders:
    count += 1
    print("count=" + str(count) + ":" + folder)
    # if folder has only one '/'
    if folder.count('/') == 1:
        # replace 'course files/' with 'files/'
        folder_name = folder.replace('course files/', '')
        print("create folder:" + folder_name)
        root_folder.create_folder(name=folder_name)
        print('    ')
    else:
        # call recursive function to create folders
        print("recursive create folder and all its parent folders:" + folder)
        create_folder_recursive(folder, root_folder)
        print('    ')

In [ ]:
# now get a list of all the folders in the target course again
target_course = canvas.get_course(target_course_id)
print(target_course)
# create a set
target_folders = set()
for folder in target_course.get_folders():
    # add folder to set
    target_folders.add(folder.full_name)
# print source_folders size
print("count of folders=" + len(target_folders))

In [ ]:
# and compare source and target folders
# put the diff in a list and sort it
# the diff list should be empty
diff_folders = list(source_folders - target_folders)
diff_folders.sort()
print("count of missing folders:" + len(diff_folders))